In [1]:
import pandas as pd
import os
import numpy as np

carpeta_script = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Scripts'
carpeta_resultados = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Resultados'
carpeta_datos = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Experimentos en clase\\Datos'


In [3]:
#Carga archivos
sell_in = pd.read_csv(os.path.join(carpeta_datos, 'sell-in.txt'), delimiter = '\t')
tb_productos = pd.read_csv(os.path.join(carpeta_datos, 'tb_productos.txt'), delimiter = '\t')
tb_stocks = pd.read_csv(os.path.join(carpeta_datos, 'tb_stocks.txt'), delimiter = '\t')
productos_predecir = pd.read_csv(os.path.join(carpeta_datos, 'productos_a_predecir.txt'), delimiter = '\t')  

In [4]:
sell_in.head()
productos_predecir.head()

ventas_producto_mes = sell_in.groupby(['periodo', 'product_id'])['tn'].sum()
ventas_producto_mes = ventas_producto_mes.reset_index()


In [5]:
ventas_producto_mes['Timestamp'] = pd.to_datetime(ventas_producto_mes['periodo'], format='%Y%m')
ventas_producto_mes.set_index('Timestamp', inplace=True)
ventas_producto_mes.drop(columns=['periodo'], inplace=True)



In [ ]:
from pmdarima.arima import auto_arima

lista_productos_arima = []
lista_predicciones_dic_arima = []
lista_valores_reales_dic = []
lista_cantidad_valores = []

for producto in ventas_producto_mes['product_id'].unique():
    if producto in list(productos_predecir['product_id']):
        try:
            ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
            ventas_mes_por_producto.drop(columns=['product_id'], inplace = True)
            valores_entrenamiento = ventas_mes_por_producto[(ventas_mes_por_producto.index != '2019-12-01') & (ventas_mes_por_producto.index != '2019-11-01')]
            valor_real_dic = ventas_mes_por_producto[ventas_mes_por_producto.index == '2019-12-01']
            modelo_autoarima = auto_arima(valores_entrenamiento, start_p=0, max_p=12, start_d=0, max_d=12, start_q=0, max_q=12, seasonal=False, trace=False, stepwise=True)
            prediccion_autoarima_mes1, prediccion_autoarima_mes2 = modelo_autoarima.predict(n_periods=2, return_conf_int=False)
            lista_productos_arima.append(producto)
            lista_predicciones_dic_arima.append(prediccion_autoarima_mes2)
            lista_valores_reales_dic.append(valor_real_dic.values[0,0])
            lista_cantidad_valores.append(valores_entrenamiento['tn'].count())
        except Exception as e:
            print(f'Error en producto {producto}')
            print(e)
            print('Imprimiendo valores de entrenamiento')
            print(valores_entrenamiento)
            lista_productos_arima.append(producto)
            lista_predicciones_dic_arima.append(valores_entrenamiento['tn'].mean())
            lista_valores_reales_dic.append(valor_real_dic.values[0,0])
            lista_cantidad_valores.append(valores_entrenamiento['tn'].count())

C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(2,0,3)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
                           ^^^^^^^^^^^^^^
  File "C:\Users\alope\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit

In [ ]:
resultados_arima = pd.DataFrame({'product_id': lista_productos_arima, 'Prediccion': lista_predicciones_dic_arima, 
                                 'Valor_dic': lista_valores_reales_dic, 'Cantidad_ej_entrenamiento': lista_cantidad_valores})

In [ ]:
resultados_arima.head()

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

# Dispersion valores dic vs prediccion
scatter = px.scatter(resultados_arima, x='Valor_dic', y='Prediccion', title="Predicción con modelo ARIMA")

# Add names to each point in the scatter plot
scatter.update_traces(
    hovertext=resultados_arima['product_id'],
    hovertemplate='<b>Producto:%{hovertext}</b><br>Valor Diciembre: %{x}<br>Predicción: %{y}<extra></extra>',
    mode='markers'
)


# Recta de pendiente 1
line_x = np.linspace(0, max(resultados_arima['Valor_dic']), 1000)
line_y = line_x

# Agregar línea al plot
scatter.add_trace(go.Scatter(x=line_x, y=line_y, mode='lines', name='Predicción=Target', line=dict(color='red', dash='dash', width=2),opacity=0.5))

#Guardar archivo HTML
scatter.write_html(os.path.join(carpeta_resultados, "Errores_ARIMA_pred_diciembre.html"))

# Show the plot
scatter.show()

In [ ]:
resultados_arima['Error_abs'] = np.abs(resultados_arima['Valor_dic'] - resultados_arima['Prediccion'])

resultados_arima.to_excel(os.path.join(carpeta_resultados, "Prediccion_ARIMA_Dic.xlsx"), index=False)

In [3]:
resultados_arima = pd.read_excel(os.path.join(carpeta_resultados, "Prediccion_ARIMA_Dic.xlsx"))

In [4]:
len(resultados_arima)

780